'''
数据集只有一个
基学习器是有依赖
获取结果的策略采用加权投票
每个学习器针对数据集训练 得到结果 计算错误率
    利用错误率给数据集中的每个样本加权(错误的权重升高，正确的权重降低)
    将加权后的样本集给下一个基学习器进行学习
    循环停止 条件 1.学习器的数量到指定值  2.错误率已经无法再提升
'''

In [1]:
from numpy import *

In [2]:
def loadDataSet():
    datMat=matrix( [[1.,2.1],
                    [2.,1.1],
                    [1.3,1.],
                    [1.,1.],
                    [2.,1.] ])
    classLabels=[1.0,1.0,-1.0,-1.0,1.0]
    return datMat,classLabels

In [3]:
dataMat,classLabels=loadDataSet()

In [4]:
#一个最简单的决策树  根据单特征来决策 所有只有一次分列过程
def simgleClassify(dataMat,featIndex,value,thredIneq):
    '''
    dataMat  待测数据集
    featIndex  判断的 特征列索引
    value : 列值
    thredIneq   数字型  所以大小关系
    '''
    resultArray=ones( (shape(dataMat)[0] ,1))
    if thredIneq=='It':
        resultArray[ dataMat[:,featIndex]<=value ]=-1.0
    else:
        resultArray[ dataMat[:,featIndex] >value ]=-1.0
    return resultArray

In [5]:
simgleClassify(dataMat,0,1,'It')

array([[-1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.]])

In [6]:
'''
算法步骤：
    将最小错误率 minError 设无穷的大
    对数据集 datMat 的每一列进行循环
            对每个步长循环
                对每个比较符号进行循环
                  利用上面三个循环产生的三个条件，调用 simgleClassify 函数 构建决策树
                     计算错误率 如果错误低于 minError 则将当前这个设置为最佳
    返回最佳决策树
'''
def buildOneEstimator(dataMat,classLabels,D,base_estimator=simgleClassify,learning_rate=0.1):
    '''
    D   权重向量  [1,1,1,1,1]
    base_estimator : 基学习器
    ''' 
    classLabels=mat(classLabels).T   #转置  变为  5行1列  与数据集对应
    bestClass={}  #树
    numSteps=10  #步长次数
    m,n=shape(dataMat)  #  m行 n 列
    bestClaEstimator = mat(zeros((m,1) )  )
    minError=inf  #最小错误率  minError设为无穷大
    #循环所有的特征
    for i in range(n):
        #取出最大值与最小值  用于生成步长
        rangeMin=dataMat[:,i].min()
        rangeMax=dataMat[:,i].max()
        #求出步长
        step=(rangeMax-rangeMin)/numSteps   #step值越大  生成的树越多
        for j in range(-1,numSteps+1):
            #循环
            for thredIneq in ['It','gt']:
                #求出分段用的value
                value=rangeMin+step*j
                resultArray=simgleClassify(dataMat,i,value,thredIneq)
                #计算这个结果的错误率
                errArr=mat(ones((m,1) )  )
                errArr[resultArray==classLabels]=0
                #将错误  向量 errArr  和权重向量  D  相乘
                weightError=D.T*errArr
                if weightError<minError:
                    minError=weightError
                    bestClaEstimator=resultArray.copy()
                    bestClass['dim']=i
                    bestClass['thresh']=value
                    bestClass['ineq']=thredIneq
    return bestClass,minError,bestClaEstimator

In [7]:
D=mat(ones((5,1)) /5 )
print(D)
buildOneEstimator(dataMat,classLabels,D)

[[ 0.2]
 [ 0.2]
 [ 0.2]
 [ 0.2]
 [ 0.2]]


({'dim': 0, 'ineq': 'It', 'thresh': 1.3}, matrix([[ 0.2]]), array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

In [8]:
'''
AdaBoosting 训练过程
步骤：
    1.对每次循环
    利用 buildOneEstimator() 函数找到最佳的单层决策树
    将最佳单层决策树加入到单层决策树数组
    计算 alpha值
    计算新的权重向量D
    计算累计类别估计值
    如果错误率等于0则退出循环
'''
def ZyAdaBoostClassifier(dataMat,classLabels,base_estimator=buildOneEstimator,n_estimator=40,learning_rate=0.1 ):
    dataMat=mat(dataMat)
    weakEstimatorArray=[]  #弱学习器的数据   他就是集成模型
    
    m,n=shape(dataMat)
    D=mat(ones((m,1) )/m )   #权重  ones( (m,1) )  D指 datmat 中每个样本 数据权重
    
    for i in range(n_estimator):
        bestEstimator,minError,result=base_estimator(dataMat,classLabels,D,simgleClassify,learning_rate)
        #根据 minError 计算这个 bestEstimator 的 alpha 的值   alpha=1/( 2 * In( (1-error)/error )  )
        alpha=float( 0.5*log((1.0-minError)/max(minError,1e-16) )  )  # max(minError,1e-16) 防止为0 出异常
        #将这个alpha值记录到 bestEstimator 这个树中  因为将来在最终结果的投票中起作用
        bestEstimator['alpha']=alpha
        weakEstimatorArray.append(bestEstimator)
        #更新样本的权重  正确样本  Di^(t+1)=(Di^t*e^-a)/sum(D)   错误样本 ：Di^(t+1)=(Di^t*e^a)/sum(D)
        expon=multiply(-1*alpha*mat(classLabels).T,result)  #这里是计算 -a 还 a
        '''
                -1 * 0.5 * 1  ->  -0.5
                -1 * 0.5 * -1  ->  0.5
        '''
        D=multiply(D,exp(expon) )
        D=D/D.sum()
        print('第',i,'个学习器更新后的样本权重:',D)
        
        #计算器错误率
        #这个 estimator 估计结果为 result  准确性由 alpha 来修正
        classResultSum=alpha*result 
        #而修正后的结果为一个float 型 但这里是一个分类模型  要转为一个分类值  这里采用
        #  sign()  函数来 解决    if x<0  -1   if x==0  0   if x>0  1
        signResult=sign(classResultSum)
        #计算错误总数
        resultErrors=multiply(signResult!=mat(classLabels).T,ones((m,1 ) ) )
        
        errorRate=resultErrors.sum()/m
        
        if errorRate==0.0:
            break
    return weakEstimatorArray

In [9]:
weakEstimatorArray=ZyAdaBoostClassifier(dataMat,classLabels)
print(len(weakEstimatorArray))

weakEstimatorArray

第 0 个学习器更新后的样本权重: [[ 0.5  ]
 [ 0.125]
 [ 0.125]
 [ 0.125]
 [ 0.125]]
第 1 个学习器更新后的样本权重: [[ 0.28571429]
 [ 0.07142857]
 [ 0.07142857]
 [ 0.07142857]
 [ 0.5       ]]
第 2 个学习器更新后的样本权重: [[ 0.16666667]
 [ 0.04166667]
 [ 0.25      ]
 [ 0.25      ]
 [ 0.29166667]]
第 3 个学习器更新后的样本权重: [[ 0.5  ]
 [ 0.025]
 [ 0.15 ]
 [ 0.15 ]
 [ 0.175]]
第 4 个学习器更新后的样本权重: [[ 0.3030303 ]
 [ 0.01515152]
 [ 0.09090909]
 [ 0.09090909]
 [ 0.5       ]]
第 5 个学习器更新后的样本权重: [[ 0.18518519]
 [ 0.00925926]
 [ 0.25      ]
 [ 0.25      ]
 [ 0.30555556]]
第 6 个学习器更新后的样本权重: [[ 0.5       ]
 [ 0.00568182]
 [ 0.15340909]
 [ 0.15340909]
 [ 0.1875    ]]
第 7 个学习器更新后的样本权重: [[ 0.30769231]
 [ 0.0034965 ]
 [ 0.09440559]
 [ 0.09440559]
 [ 0.5       ]]
第 8 个学习器更新后的样本权重: [[ 0.18965517]
 [ 0.00215517]
 [ 0.25      ]
 [ 0.25      ]
 [ 0.30818966]]
第 9 个学习器更新后的样本权重: [[ 0.5       ]
 [ 0.00132979]
 [ 0.15425532]
 [ 0.15425532]
 [ 0.19015957]]
第 10 个学习器更新后的样本权重: [[ 0.30870279]
 [ 0.00082102]
 [ 0.0952381 ]
 [ 0.0952381 ]
 [ 0.5       ]]
第 11 个学习器更新后的样本

[{'alpha': 0.6931471805599453, 'dim': 0, 'ineq': 'It', 'thresh': 1.3},
 {'alpha': 0.9729550745276565, 'dim': 1, 'ineq': 'It', 'thresh': 1.0},
 {'alpha': 0.8958797346140273,
  'dim': 0,
  'ineq': 'It',
  'thresh': 0.90000000000000002},
 {'alpha': 0.8047189562170499, 'dim': 0, 'ineq': 'It', 'thresh': 1.3},
 {'alpha': 0.7752987062055835, 'dim': 1, 'ineq': 'It', 'thresh': 1.0},
 {'alpha': 0.752038698388137,
  'dim': 0,
  'ineq': 'It',
  'thresh': 0.90000000000000002},
 {'alpha': 0.7408022704621077, 'dim': 0, 'ineq': 'It', 'thresh': 1.3},
 {'alpha': 0.7331685343967135, 'dim': 1, 'ineq': 'It', 'thresh': 1.0},
 {'alpha': 0.7288766625510178,
  'dim': 0,
  'ineq': 'It',
  'thresh': 0.90000000000000002},
 {'alpha': 0.726126164455844, 'dim': 0, 'ineq': 'It', 'thresh': 1.3},
 {'alpha': 0.7244868187249948, 'dim': 1, 'ineq': 'It', 'thresh': 1.0},
 {'alpha': 0.7234594914681627,
  'dim': 0,
  'ineq': 'It',
  'thresh': 0.90000000000000002},
 {'alpha': 0.7228333173099297, 'dim': 0, 'ineq': 'It', 'thresh

In [10]:
#最后将集成模型整合到一个函数里面
'''
循环这个基学习器  用它来对dataMat 做一次预测  simgleClassify()  得到一个预测的结果集
再用这个结果加权后求和
再sign后得到的结果
'''
def adaClassify(dataMat,weakEstimatorArray):
    '''
    dataMat  一个或多个待分类
    weakEstimatorArray  多个弱分类器组成的数组
    '''
    dataMat=mat(dataMat)
    m,n=shape(dataMat)
    resultSum=mat(zeros((m,1)))
    for es in weakEstimatorArray:
        result=simgleClassify(dataMat,es['dim'],es['thresh'],es['ineq'])
        print(result,' ',es['alpha'])
        resultSum+=es['alpha']*result
    return sign(resultSum)
